In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Import Data

In [2]:
mnist_dataset, mnist_info = tfds.load('mnist', as_supervised = True, with_info = True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

## Preprocess

In [3]:
# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
# this is also where we make use of mnist_info (we don't have to count the observations)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# let's cast this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
?tf.cast

In [5]:
def scale(image, label):
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.
    return image, label

In [6]:
?mnist_test.map

In [7]:
# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

# there is no need to shuffle test data because we won't be training on it
test_data = mnist_test.map(scale)

In [8]:
BUFFER_SIZE = 10000 
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [9]:
?shuffled_train_and_validation_data.take

In [10]:
?shuffled_train_and_validation_data.skip

In [11]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples) #10%
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) #Rest

In [12]:
?validation_data.batch

In [13]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
#Model now expects test data in batch form too
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

## Model

In [14]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(output_size, activation='softmax')
])

## Loss & Optimization

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

## Training Model

In [20]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2, 
          validation_steps = 1)

Epoch 1/5
540/540 - 5s - loss: 0.0504 - accuracy: 0.9850 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 6s - loss: 0.0437 - accuracy: 0.9874 - val_loss: 0.0568 - val_accuracy: 0.9820
Epoch 3/5
540/540 - 6s - loss: 0.0421 - accuracy: 0.9872 - val_loss: 0.0620 - val_accuracy: 0.9807
Epoch 4/5
540/540 - 6s - loss: 0.0368 - accuracy: 0.9889 - val_loss: 0.0496 - val_accuracy: 0.9850
Epoch 5/5
540/540 - 6s - loss: 0.0336 - accuracy: 0.9894 - val_loss: 0.0493 - val_accuracy: 0.9865


## Testing Model

In [21]:
test_loss, test_accuracy = model.evaluate(test_data)
test_loss

1/1 [==============================] 0.1033 - accuracy: 0.97 - 1s 1s/step - loss: 0.1033 - accuracy: 0.9701


0.10330205410718918

In [22]:
test_accuracy

0.9701